# Setting up your Edge device

## Summary 

This notebook explains how to register your DSVM as an edge device and enable it as a gateway. 

## Description

The purpose of this notebook is to run trough and explain the scripts that have been included in the DSVM IoT extension that register and enable the VM as an Edge device.

## Requirements

* Before starting this notebook you need to perform an azure cli login (az login)


## Documentation
https://docs.microsoft.com/en-us/azure/iot-edge/how-to-create-transparent-gateway-linux

https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-linux

## Step 1: Enable the VM as an Edge device

The DSVM IoT extension includes by default the Iot Edge runtime, it also includes an script for registering the VM as an Edge Device to the IoT hub that was selected during the creation of the Extension.

The path to this script is /home/$USER/IotEgde/edge_configure.sh

The script has this steps:

1. Retrieves the Iot Hub name that was selected during the creation of the VM and also the subscription Id. 
2. Performs an Az login in order to verify the user.
3. Creates a new Edge device on IoT Hub and retrieves the connection string. 
4. Inserts the connection string into the Iot Edge daemon config file.
5. Restarts the service. 

As soon as you run this script, you're able to specify deployments for your VM from Iot Hub.


In [ ]:
%%bash
source /etc/profile
# Obtaining Vm info
user="$(whoami)"
hubName="$(cat /home/$user/.hubName.txt)"
host="$(hostname)"
device="${host}_edge"
subid="$(cat /home/$user/.vminfo.json  | jq '.compute.subscriptionId')"
subid="${subid%\"}"
subid="${subid#\"}"
az account set -s $subid
# Register edge device 
az iot hub device-identity create --device-id $device --hub-name $hubName --edge-enabled
# Inserting connection string into config file
connection_string="$(az iot hub device-identity show-connection-string --device-id $device --hub-name $hubName )"
connection_string="${connection_string::-2}"
connection_string="${connection_string:10}"
connection_string="  device_connection_string: $connection_string"
sudo sed -i '26s/.*//'  /etc/iotedge/config.yaml
sudo sed -i '26i\'"$connection_string" /etc/iotedge/config.yaml
# Reboting edge for pairing to proceed
sudo systemctl restart iotedge

## Step 2: Enable the VM as a Gateway

You can send telemetry data generated by IoT devices directly into the DSVM by enabling the Iot Edge runtime as a gateway. We have added an script in order to enable the gateway in a quickstart scenario.

The path to this script is /home/$USER/IotEdge/gateway/gateway_configure.sh

Note: you need to send as a parameter a name for the device, this is used in order to generate the certificates. 

The script has this steps:

1. It validates that you have send the device name.
2. Generates the quickstart certificates.
3. Generates the certificate chain.
4. Inserts the certificates location into the Iot Edge Daemon config file.
5. Restarts the service.

Note: 
    * The scripts for generating the certificates are not meant for production eviroments, just for a quickstart. 
    * DO NOT use a name that is the same as the gateway's DNS host name. Doing so will cause client certification against the certificates to fail.


In [ ]:
%%bash
source /etc/profile
## You need to set your device name for the certificates
device_name="mytestvm"
if [ -z ${device_name+x} ];
then
    echo "Missing parameter: Gateway device name"
    exit 1
fi
directory="/home/$USER/IoT/IotEdge/gateway"
# Certificate generation
(cd $directory/certificates/; sudo bash certGen.sh create_root_and_intermediate; sudo bash certGen.sh create_edge_device_certificate $device_name )
#(cd certificates/; sudo bash certGen.sh create_edge_device_certificate $device_name)
# Chain creation
sudo chmod 777 $directory/certificates/certs
sudo cat $directory/certificates/certs/new-edge-device.cert.pem $directory/certificates/certs/azure-iot-test-only.intermediate.cert.pem $directory/certificates/certs/azure-iot-test-only.root.ca.cert.pem > $directory/certificates/certs/new-edge-device-full-chain.cert.pem
certdir="$directory/certificates"
device_ca_cert="  device_ca_cert: $certdir/certs/new-edge-device-full-chain.cert.pem"
device_ca_pk="  device_ca_pk: $certdir/private/new-edge-device.key.pem"
trusted_ca_certs="  trusted_ca_certs: $certdir/certs/azure-iot-test-only.root.ca.cert.pem"
title="certificates:"
sudo sed -i '53s/.*//'  /etc/iotedge/config.yaml
sudo sed -i '54s/.*//'  /etc/iotedge/config.yaml
sudo sed -i '55s/.*//'  /etc/iotedge/config.yaml
sudo sed -i '56s/.*//'  /etc/iotedge/config.yaml
sudo sed -i '53i\'"$title" /etc/iotedge/config.yaml
sudo sed -i '54i\'"$device_ca_cert" /etc/iotedge/config.yaml
sudo sed -i '55i\'"$device_ca_pk" /etc/iotedge/config.yaml
sudo sed -i '56i\'"$trusted_ca_certs" /etc/iotedge/config.yaml
sudo systemctl restart iotedge

## [Optional] Step 3: Troubleshooting the Gateway:


At the moment of writing this notebook, the IoT Edge gateway presents an issue where the runtime does not uses the user generated certificates and instead it uses the 'Quick Start' certificates which does not allow us to connect to the gateway, in order to test that everything is set up right run the following command, if everything is right you should see a 'Verified ok' status if not, then follow the next steps

In [ ]:
%%bash
openssl s_client -connect localhost:8883 -CAfile /home/$USER/IoT/IotEdge/gateway/certificates/certs/azure-iot-test-only.root.ca.cert.pem -showcerts

If the last commands failed, then we are going to run the following commands, what will happen is that it will delete the 'Quick Start' certificates folders, the docker images and then restart the service for it to work.

In [ ]:
%%bash

sudo systemctl stop iotedge
sudo rm -rf /var/lib/iotedge/hsm/certs
sudo rm -rf /var/lib/iotedge/hsm/cert_keys
sudo docker rmi mcr.microsoft.com/azureiotedge-hub:1.0 -f
sudo docker rmi mcr.microsoft.com/azureiotedge-agent:1.0 -f
sudo systemctl restart iotedge

